In [ ]:
# ! pip install --upgrade ipywidgets

In [7]:
import pandas as pd
from utils import *
from sklearn.ensemble import RandomForestClassifier
from tqdm.notebook import trange, tqdm
from time import sleep
from component_store import GeneticAlgorithmComponent
import numpy as np

features = best_features

# Hyper Parameter Tuning

In [ ]:

perf = pd.DataFrame(columns=["lookback", "train_start", "train_end", "test_year", "inds", "train_score", "test_score"])

In [ ]:
get_training_data([2023]).columns[:22]

In [ ]:
# features = [0, 2, 3, 4]
# features = [5, 6, 8, 10, 12, 14, 16, 18, 20]
# features = [4, 7, 9, 11, 13, 15, 17, 19, 21]


In [ ]:
column_selector(get_training_data([2023]), features)

In [ ]:
get_training_data([2023]).columns

In [ ]:
def run_pipe(train_start, train_end, test_year, features):
    train_df = get_training_data([train_start + x for x in range(train_end-train_start+1)])
    train_X = column_selector(train_df, features)
    train_y = train_df['result']

    if train_df.size == 0:
        return

    clf = RandomForestClassifier()
    clf.fit(train_X, train_y)

    test_df = get_training_data([test_year])
    test_X = column_selector(test_df, features)
    test_y = test_df['result']

    # print(test_X)
    # print(test_y)
    # print(clf.predict(test_X))
    # print(clf.predict_proba(test_X))

    if test_df.size == 0:
        return

    train_score = clf.score(train_X, train_y)
    test_score = clf.score(test_X, test_y)

    return train_score, test_score, clf

In [ ]:
train_score, test_score, _ = run_pipe(2019, 2022, 2023, features)
train_score, test_score

In [ ]:

for l in trange(1):
    # print("lookback")    
    lookback = l + 5
    for ty in trange(24-3 - lookback):
        # print(f"test_year: {test_year}")
        test_year = 2003 + lookback + ty
        for feature in range(1):
            # print(f"feature: {feature}")
            train_start, train_end = test_year - lookback, test_year - 1

            # print(lookback, train_start, train_end, test_year)
            # sleep(0.1)

            results = run_pipe(train_start, train_end, test_year, features)

            if results is None:
                continue
            train_score, test_score, _ = results
            
            perf = pd.concat((
                perf, 
                pd.DataFrame([[
                    lookback, train_start, train_end, test_year,
                    str(features), 
                    train_score, 
                    test_score
                ]], columns=["lookback", "train_start", "train_end", "test_year", "inds", "train_score", "test_score"])
            ), axis=0)

In [ ]:
perf

In [ ]:
grouped = perf[["lookback", "inds", "train_score", "test_score"]].groupby(by=["lookback", "inds"]).mean().sort_values(by="test_score", ascending=False)
grouped = grouped["test_score"].reset_index().pivot(index="lookback", columns="inds", values="test_score")
grouped

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(grouped.index, grouped.iloc[:, 0])
plt.show

In [ ]:
grouped.loc[grouped.mean(axis=1).sort_values(ascending=False).index, grouped.mean(axis=0).sort_values(ascending=False).index]

In [ ]:
perf

# Genetic Algo

In [8]:
results = run_pipe(2013, 2018, 2019, features)
train_acc, test_acc, clf = results
results

(1.0, 0.7238805970149254, RandomForestClassifier())

In [9]:
features

[5, 6, 8, 10, 12, 14, 16, 18, 20]

In [10]:

test_df = get_training_data([2019])
test_X = column_selector(test_df, features)
test_y = test_df['result']
test_X

,A_ADJEM,A_ADJOE,A_ADJDE,A_ADJ_T,A_Luck,A_SOS,A_OppO,A_OppD,A_NCSOS,B_ADJEM,B_ADJOE,B_ADJDE,B_ADJ_T,B_Luck,B_SOS,B_OppO,B_OppD,B_NCSOS
1112,-4.13,102.1,106.2,71.1,0.061,-7.85,98.8,106.7,6.15,-4.22,106.5,110.7,66.8,0.040,-8.16,99.5,107.7,-1.86
1113,11.55,109.7,98.2,70.8,0.051,5.56,107.3,101.7,1.55,7.61,108.6,101.0,70.8,0.060,5.00,106.9,101.9,-6.49
1114,-3.20,107.4,110.6,65.0,0.064,-2.85,104.8,107.6,3.23,-11.55,97.8,109.3,65.3,-0.008,-11.84,97.0,108.9,1.19
1115,14.09,115.6,101.5,71.3,0.025,-1.98,103.6,105.6,3.78,9.72,109.2,99.5,68.9,0.064,4.55,106.3,101.8,-1.07
1116,14.89,104.3,89.5,68.4,0.007,1.37,105.2,103.8,3.60,16.51,112.1,95.6,65.0,0.011,4.99,105.9,100.9,-3.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2554,30.03,114.1,84.1,66.6,0.004,11.18,109.8,98.7,-5.39,32.85,124.5,91.6,70.2,-0.001,4.46,106.9,102.5,1.87
2555,26.81,122.5,95.6,65.9,-0.015,13.16,110.3,97.1,4.09,34.22,123.4,89.2,59.4,0.050,11.18,109.2,98.1,-3.24
2556,25.00,120.9,95.9,67.9,0.008,12.23,110.7,98.5,2.63,34.22,123.4,89.2,59.4,0.050,11.18,109.2,98.1,-3.24
2557,30.03,114.1,84.1,66.6,0.004,11.18,109.8,98.7,-5.39,30.81,121.0,90.2,66.9,0.001,13.67,110.6,96.9,3.24


In [11]:
test_year = 2019
df = pd.read_csv("../data/cleaned_bracket_data.csv", index_col=0)
df = df[(df["year"] == test_year) & (df["round"] == 1)]
start_team_names = df[["team1", "team2"]].reset_index().melt(id_vars=['index'], value_vars=['team1', 'team2']).sort_values(["index", "variable"]).reset_index(drop=True)["value"]
start_team_names = pd.merge(start_team_names, df[["team1", "team1seed"]], how="left", left_on="value", right_on="team1").rename({"team1seed":"SEED"}, axis=1).drop("team1", axis=1)
start_team_names = pd.merge(start_team_names, df[["team2", "team2seed"]], how="left", left_on="value", right_on="team2").drop("team2", axis=1)
start_team_names.loc[start_team_names["SEED"].isna(), "SEED"] = start_team_names["team2seed"]
start_team_names = start_team_names.drop("team2seed", axis=1)

# features_df = pd.read_csv("../data/kenpom.csv", index_col=0)
# features_df = features_df[features_df["YEAR"] == test_year]

# teams_df = pd.merge(start_team_names, features_df, how="left", left_on="value", right_on="TEAM")
# teams_df = pd.concat((teams_df.iloc[:, -3:], teams_df.iloc[:, 1:2], teams_df.iloc[:, 2:3], teams_df.iloc[:, 6:-4]), axis=1)
# # stats_2 = pd.concat((df_2.iloc[:, -4:], df_2.iloc[:, 3:4], df_2.iloc[:, 7:-5]), axis=1)
# teams_df_1 = teams_df.iloc[0::2].add_prefix("A_").reset_index(drop=True)
# teams_df_2 = teams_df.iloc[1::2].add_prefix("B_").reset_index(drop=True)
# teams_df = pd.concat((teams_df_1, teams_df_2), axis=1)
# print(teams_df.shape)
# X_df = column_selector(teams_df, features)

# X_df = make_X_from_teams(start_team_names, test_year, features)

# pred_probs = clf.predict_proba(X_df)
# r = np.random.rand(pred_probs.shape[0])
# r = (pred_probs[:, 0] < r).astype(int)
# pred_inds = np.arange(pred_probs.shape[0]) * 2 + r
# next_team_names = start_team_names.iloc[pred_inds]

In [12]:
generate_brackets_np(clf, features, start_team_names, test_year, 10)

KeyboardInterrupt: 

In [ ]:
start_team_names

In [ ]:
seed_counts = {k:0 for k in [i+1 for i in range(16)]}
seed_counts

In [ ]:
all_brackets = np.zeros((0,63))
for i in range(100):
    bracket = np.zeros((0,))
    team_names = start_team_names

    while team_names.shape[0] > 1:
        team_names, results = run_round_np(team_names)
        bracket = np.concatenate([bracket, results])
    seed_counts[int(team_names["SEED"].iloc[0])] += 1
    # print(bracket)
    all_brackets = np.concatenate([all_brackets, bracket.reshape((1, -1))], axis=0)
all_brackets.shape

In [ ]:
from time import perf_counter

In [ ]:
num_trials = 1000
bracket_timer = 0

for i in range(num_trials):
    team_names = start_team_names
    t0 = perf_counter()
    while team_names.shape[0] > 1:
        team_names, results = run_round_np(team_names)
    t1 = perf_counter()
    bracket_timer += t1 - t0
    seed_counts[int(team_names["SEED"].iloc[0])] += 1
    
print(f"Average time of {bracket_timer / num_trials} s per bracket generated!\nDistribution of winners:\n")
seed_counts

In [ ]:
gac = GeneticAlgorithmComponent(10, 10)
population = gac.execute({"model": clf}, init_pop, )

In [ ]:
train_acc, test_acc, clf = results

In [ ]:
perf[perf["inds"] == "[5, 6, 8, 10, 12, 14, 16, 18, 20, 22]"]

In [ ]:
# [5, 6, 8, 10, 12, 14, 16, 18, 20, 22, 21

In [ ]:
grouped.mean(axis=0).sort_values(ascending=False)

In [ ]:
grouped.mean(axis=1).sort_values(ascending=False)

In [ ]:
get_training_data([2023]).columns[6:9]

In [ ]:
perf.sort_values(by="test_score", ascending=False)

In [ ]:
train_start = 2002
train_end = 2022
test_year = 2023
features = [x+1 for x in range(21)]
results = pd.DataFrame()

for i in trange(21):
    train_df = get_training_data([train_start + x for x in range(train_end-train_start+1)])
    train_X = column_selector(train_df, features)
    train_y = train_df['result']



    clf = RandomForestClassifier()
    clf.fit(train_X, train_y)

    test_df = get_training_data([test_year])
    test_X = column_selector(test_df, features)
    test_y = test_df['result']

    # print(test_X)
    # print(test_y)
    # print(clf.predict(test_X))
    # print(clf.predict_proba(test_X))


    train_score = clf.score(train_X, train_y)
    test_score = clf.score(test_X, test_y)

    print(train_score, test_score)
    importances = pd.DataFrame(clf.feature_importances_, columns=["importances"], index=features+features).reset_index().groupby("index").mean().sort_values(by="importances")
    results = pd.concat((results, pd.DataFrame([[worst, train_score, test_score]])))
    worst = importances.index[0]
    features.remove(worst)

In [ ]:
results

In [ ]:
features

In [ ]:
worst = pd.DataFrame(clf.feature_importances_, columns=["importances"], index=features+features).reset_index().groupby("index").mean().sort_values(by="importances").index[0]

In [ ]:
features.remove(worst)

In [ ]:
features